In [5]:
import re
from docx import Document
doc=Document('thesis1.docx')
text=" ".join([p.text for p in doc.paragraphs])

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  
    text = re.sub(r"\s+", " ", text)
    return text.strip()
cleaned_text = clean_text(text)
sentences = cleaned_text.split(".") 
words = cleaned_text.split()

preprocessing data

In [7]:
window_size = 5
X = []
y = []
for i in range(window_size, len(words)):
    X.append(words[i-window_size:i])
    y.append(words[i])

tokens[0]

In [8]:
from collections import Counter

import torch



counter = Counter(words)
vocab = ["<pad>", "<unk>"] + list(counter.keys())
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}

vocab_size = len(vocab)
X_idx = [
    [word2idx.get(w, word2idx["<unk>"]) for w in seq]
    for seq in X
]

y_idx = [word2idx.get(w, word2idx["<unk>"]) for w in y]

X = torch.tensor(X_idx)
y = torch.tensor(y_idx)

convert each word to unique number

In [9]:
from gensim.models import FastText
fasttext_model = FastText.load_fasttext_format('cc.fa.300.bin')
embed_dim = 300
embedding_matrix = torch.zeros((vocab_size, embed_dim))
for word, idx in word2idx.items():
  if word in fasttext_model.wv:
    embedding_matrix[idx] = torch.tensor(fasttext_model.wv[word])
    embedding_matrix=embedding_matrix.cuda()
  else:
    embedding_matrix[idx] = torch.randn(embed_dim)
    


C:\Users\USER\AppData\Local\Temp\ipykernel_62152\2279556815.py:2: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fasttext_model = FastText.load_fasttext_format('cc.fa.300.bin')


In [10]:
import torch.nn as nn
import torch.nn as nn
embedding_layer=nn.Embedding.from_pretrained(embedding_matrix,freeze=False)

In [19]:
import torch
import torch.nn as nn

class NextWordBiLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, vocab_size,dropout=0.5):
        super().__init__()


        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=True 
        )

        # مقدار embedding_dim رو از nn.Embedding بدست میاریم
        embedding_dim = self.embedding.embedding_dim

    
        self.bilstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True,
            dropout=dropout,
            
        )
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
     
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)

    def forward(self, x):
        x = self.embedding(x)  # (B, T, embedding_dim)
        out, _ = self.bilstm(x)  # (B, T, 2*hidden_dim)
        out = out[:, -1, :]  
        out = self.fc(out)  # (B, vocab_size)
        return out

In [20]:
class EarlyStopping:
    def __init__(self, patience=15, min_delta=0.01):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# In training loop
early_stopping = EarlyStopping(patience=15)

for epoch in range(300):
    # Training
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        # ... training code ...
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break

Epoch 1, Train Loss: 4.2268, Val Loss: 6.0741
Epoch 11, Train Loss: 6.7748, Val Loss: 6.0741
Early stopping at epoch 16


In [21]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [18]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X.cpu(), y.cpu(), test_size=0.2, random_state=42
)
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
model = NextWordBiLSTM(
    embedding_matrix,
    hidden_dim=128,
    vocab_size=vocab_size
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

X = X.to(device)
y = y.to(device)
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)
for epoch in range(200):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

C:\Users\USER\AppData\Local\Temp\ipykernel_62152\2833679165.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embedding_matrix, dtype=torch.float32),


TypeError: float() argument must be a string or a real number, not 'builtin_function_or_method'

In [17]:
import torch
import torch.nn.functional as F
import numpy as np
def sample_top_k(output, top_k=5, temperature=0.8):

    logits = output.squeeze() / temperature
    probs = F.softmax(logits, dim=-1).cpu().numpy()
    

    top_indices = probs.argsort()[-top_k:]
    top_probs = probs[top_indices]
    top_probs = top_probs / top_probs.sum()  # نرمال‌سازی
    chosen_idx = np.random.choice(top_indices, p=top_probs)
    
    return chosen_idx

In [14]:
def generate_text(model, seed_text, next_words=50, seq_length=10,
                  top_k=5, temperature=0.8, device='cuda'):

    model.eval()
    with torch.no_grad():
      val_outputs = model(X_val)
      val_loss = criterion(val_outputs, y_val)
      print(f"Final Validation Loss: {val_loss.item():.4f}")
    words = seed_text.split()

    for _ in range(next_words):
        # استفاده از آخرین seq_length کلمه
        input_seq = [word2idx.get(w, word2idx["<unk>"]) for w in words[-seq_length:]]
        
        # اگر seed کوتاه بود، با <pad> پر می‌کنیم
        if len(input_seq) < seq_length:
            input_seq = [word2idx["<pad>"]] * (seq_length - len(input_seq)) + input_seq
        
        input_seq = torch.tensor(input_seq).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_seq)
            predicted_idx = sample_top_k(output, top_k=top_k, temperature=temperature)
            predicted_word = idx2word.get(predicted_idx, "<unk>")

        if len(words) > 1 and predicted_word == words[-1]:
            continue  
        
        words.append(predicted_word)
    
    # ✅ Return the generated text
    return " ".join(words)



seeds = [
    "تحلیل و شناسایی الگوهای",
    "در این پژوهش به",
    "روش تحقیق این"
]

for seed in seeds:
    text = generate_text(model, seed_text=seed, next_words=30, 
                         seq_length=5, top_k=5, temperature=0.8, device=device)
    print(f"\nSeed: {seed}")
    print(f"Generated: {text}\n")
print(text)


Final Validation Loss: 6.0726

Seed: تحلیل و شناسایی الگوهای
Generated: تحلیل و شناسایی الگوهای پویا و در این حوزه و سپس تحلیل و روشهای پویا در تشخیص احساسات در این حوزه از کلاسهای و تحلیل و برای تشخیص احساسات در طول زمان و دقت

Final Validation Loss: 6.0726

Seed: در این پژوهش به
Generated: در این پژوهش به بررسی داده است که مدل با دقت در طول زمان به خوبی در طول زمان و تعداد اپوک و شناسایی و پیشبینی و شناسایی دادههای پویا را فراهم

Final Validation Loss: 6.0726

Seed: روش تحقیق این
Generated: روش تحقیق این فصل تحقیقات و در این حوزه و سپس به دلیل سادگی برای در طول زمان است در طول زمان آموزش و دقت مدل در طول زمان است که

روش تحقیق این فصل تحقیقات و در این حوزه و سپس به دلیل سادگی برای در طول زمان است در طول زمان آموزش و دقت مدل در طول زمان است که
